In [ ]:
Code cell <undefined>
#%% [code]
import tensorflow.keras, os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import zipfile

Code cell <undefined>
#%% [code]
path = os.getcwd().split("code")[0]
dataset_name = "dogs-vs-cats"
unzip = False

def unzipDataSet(zipfile_name, directory):
    with zipfile.ZipFile(f"{path}/dataset/{dataset_name}/{zipfile_name}", "r") as z:
        z.extractall(f"{path}/dataset/images/{directory}")

def createDataFrameFromDirectory(directory):
    list_of_files = [os.path.join(f"{path}/dataset/images/{directory}", file) for file in os.listdir(f"{path}/dataset/images/{directory}")]

    df = pd.DataFrame({"Path" : list_of_files})
    df["animal"] = np.where(df["Path"].str.contains("dog"), "dog", "cat")

    return df

Code cell <undefined>
#%% [code]
def setModel():
    model = Sequential()

    model.add(Conv2D(input_shape = (64,64, 3), filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))

    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096, activation="relu"))
    model.add(Dense(units=4096, activation="relu"))
    model.add(Dense(units=2, activation="softmax"))

    return model

Code cell <undefined>
#%% [code]
def run():
    if unzip:
        unzipDataSet("train.zip", "train")
        unzipDataSet("test1.zip", "test")

    df_train = createDataFrameFromDirectory("train/train")
    df_test = createDataFrameFromDirectory("test/test1")

    X_Train, X_Test = train_test_split(df_train, test_size=0.3)

    print(X_Train.shape)

    tr_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

    ts_datagen = ImageDataGenerator(rescale = 1./255)

    train_set = tr_datagen.flow_from_dataframe(dataframe=X_Train, x_col="Path", y_col="animal", class_mode="categorical", target_size=(64,64), batch_size=128)
    test_set  = ts_datagen.flow_from_dataframe(dataframe=X_Test,  x_col="Path", y_col="animal", class_mode="categorical", target_size=(64,64), batch_size=128)

    opt = Adam(learning_rate=0.001)
    model = setModel()

    model.compile(optimizer = opt, loss=categorical_crossentropy, metrics=["accuracy"])

    model.summary()

    checkpoint = ModelCheckpoint(f"{path}/models/vgg16_1.h5", monitor="accuracy", verbose=1, save_best_only=True, save_weights_only=False, mode="auto", save_freq=1)
    early = EarlyStopping(monitor="val_accuracy", min_delta=0, patience=20, verbose=1, mode="auto")

    hist = model.fit_generator(steps_per_epoch=1, generator=train_set, validation_data=test_set, validation_steps=10, epochs=1, callbacks=[checkpoint, early])
    
    return hist

Code cell <undefined>
#%% [code]
hist = run()
Execution output
5KB
	Stream
		(17500, 2)
		Found 17500 validated image filenames belonging to 2 classes.
		Found 7500 validated image filenames belonging to 2 classes.
		Model: "sequential_10"
		_________________________________________________________________
		 Layer (type)                Output Shape              Param #   
		=================================================================
		 conv2d_130 (Conv2D)         (None, 64, 64, 64)        1792      
		                                                                 
		 conv2d_131 (Conv2D)         (None, 64, 64, 64)        36928     
		                                                                 
		 max_pooling2d_50 (MaxPoolin  (None, 32, 32, 64)       0         
		 g2D)                                                            
		                                                                 
		 conv2d_132 (Conv2D)         (None, 32, 32, 128)       73856     
		                                                                 
		 conv2d_133 (Conv2D)         (None, 32, 32, 128)       147584    
		                                                                 
		 max_pooling2d_51 (MaxPoolin  (None, 16, 16, 128)      0         
		 g2D)                                                            
		                                                                 
		 conv2d_134 (Conv2D)         (None, 16, 16, 256)       295168    
		                                                                 
		 conv2d_135 (Conv2D)         (None, 16, 16, 256)       590080    
		                                                                 
		 conv2d_136 (Conv2D)         (None, 16, 16, 256)       590080    
		                                                                 
		 max_pooling2d_52 (MaxPoolin  (None, 8, 8, 256)        0         
		 g2D)                                                            
		                                                                 
		 conv2d_137 (Conv2D)         (None, 8, 8, 512)         1180160   
		                                                                 
		 conv2d_138 (Conv2D)         (None, 8, 8, 512)         2359808   
		                                                                 
		 conv2d_139 (Conv2D)         (None, 8, 8, 512)         2359808   
		                                                                 
		 max_pooling2d_53 (MaxPoolin  (None, 4, 4, 512)        0         
		 g2D)                                                            
		                                                                 
		 conv2d_140 (Conv2D)         (None, 4, 4, 512)         2359808   
		                                                                 
		 conv2d_141 (Conv2D)         (None, 4, 4, 512)         2359808   
		                                                                 
		 conv2d_142 (Conv2D)         (None, 4, 4, 512)         2359808   
		                                                                 
		 max_pooling2d_54 (MaxPoolin  (None, 2, 2, 512)        0         
		 g2D)                                                            
		                                                                 
		 flatten_10 (Flatten)        (None, 2048)              0         
		                                                                 
		 dense_30 (Dense)            (None, 4096)              8392704   
		                                                                 
		 dense_31 (Dense)            (None, 4096)              16781312  
		                                                                 
		 dense_32 (Dense)            (None, 2)                 8194      
		                                                                 
		=================================================================
		Total params: 39,896,898
		Trainable params: 39,896,898
		Non-trainable params: 0
		_________________________________________________________________
		/Users/ivastival/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
		WARNING:tensorflow:6 out of the last 13 calls to <function Model.make_train_function.<locals>.train_function at 0x7f913ee94510> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
		
		Epoch 00001: accuracy improved from -inf to 0.42969, saving model to /Users/ivastival/OneDrive/Developer/DataScience/TranferLearning//models/vgg16_1.h5
		1/1 [==============================] - 73s 73s/step - loss: 0.6933 - accuracy: 0.4297 - val_loss: 0.7010 - val_accuracy: 0.5109

Code cell <undefined>
#%% [code]
for key in hist.history:
    print(key)
Execution output
0KB
	Stream
		loss
		accuracy
		val_loss
		val_accuracy

Code cell <undefined>
#%% [code]


